In [9]:
import pandas as pd
cols = [
    'id',
    'property_type',
    'room_type',
    'amenities',
    'accommodates',
    'bathrooms',
    'bed_type',
    'cancellation_policy',
    'cleaning_fee',
    'city',
    'description',
    'first_review',
    'host_has_profile_pic',
    'host_identity_verified',
    'host_response_rate',
    'host_since',
    'instant_bookable',
    'last_review',
    'latitude',
    'longitude',
    'name',
    'neighbourhood',
    'number_of_reviews',
    'review_scores_rating',
    'thumbnail_url',
    'zipcode',
    'bedrooms',
    'beds',
    'price']
datalatih = pd.read_csv('D:/S2 IPB/Semester 2/Pembelajaran Mesin Statistika/UTS/train.csv', usecols=cols)
datalatih.shape[0]

51879

In [10]:
import numpy as np
# Menghitung berapa banyak nilai yang hilang tiap kolom
print(datalatih.isnull().sum())

id                            0
property_type                 0
room_type                     0
amenities                     0
accommodates                  0
bathrooms                   144
bed_type                      0
cancellation_policy           0
cleaning_fee                  0
city                          0
description                   0
first_review              11014
host_has_profile_pic        128
host_identity_verified      128
host_response_rate        12754
host_since                  128
instant_bookable              0
last_review               10989
latitude                      0
longitude                     0
name                          0
neighbourhood              4774
number_of_reviews             0
review_scores_rating      11621
thumbnail_url              5765
zipcode                     670
bedrooms                     66
beds                         84
price                         0
dtype: int64


In [11]:
# kolom yang memiliki banyak nilai hilang lebih baik dihilangkan saja
# kolom atau variabel yang di hilangkan first_review, host_response_rate, last_review, thumbnail_url, neighbourhood
# Kecuali review_scores_rating
data = datalatih.drop(columns=['description','first_review','host_response_rate','last_review','name','thumbnail_url'])

In [12]:
# Asumsi bahwa nilai NaN pada kolom review score rating karena memang belum ada penilaian
# sehinggan nilai NaN akan diubah menjadi nol
data['review_scores_rating'] = data['review_scores_rating'].fillna(0)
print(data.isnull().sum())

id                           0
property_type                0
room_type                    0
amenities                    0
accommodates                 0
bathrooms                  144
bed_type                     0
cancellation_policy          0
cleaning_fee                 0
city                         0
host_has_profile_pic       128
host_identity_verified     128
host_since                 128
instant_bookable             0
latitude                     0
longitude                    0
neighbourhood             4774
number_of_reviews            0
review_scores_rating         0
zipcode                    670
bedrooms                    66
beds                        84
price                        0
dtype: int64


In [13]:
# Akan dihilangkan baris yang memiliki NaN
data = data.dropna()
data.shape[0]

46350

In [14]:
# Mengubah kolom menjadi kategorik
data.host_has_profile_pic = data.host_has_profile_pic.replace({"t": 1, "f": 0})
data.host_identity_verified  = data.host_identity_verified.replace({"t": 1, "f": 0})
data.cleaning_fee  = data.cleaning_fee.replace({True: 1, False: 0})
data.instant_bookable   = data.instant_bookable.replace({"t": 1, "f": 0})
# Menghilangkan kolom id karena tidak akan digubakan dalam pemodelan
df = data.drop('id', axis=1)

In [15]:
# Mengubah format date hosting since
from datetime import datetime
now = datetime.now()
now = now.strftime("%m/%d/%Y")

# Menbuat kolom baru waktu sekarang
df['days'] = now
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46350 entries, 0 to 51878
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   property_type           46350 non-null  object 
 1   room_type               46350 non-null  object 
 2   amenities               46350 non-null  object 
 3   accommodates            46350 non-null  int64  
 4   bathrooms               46350 non-null  float64
 5   bed_type                46350 non-null  object 
 6   cancellation_policy     46350 non-null  object 
 7   cleaning_fee            46350 non-null  int64  
 8   city                    46350 non-null  object 
 9   host_has_profile_pic    46350 non-null  int64  
 10  host_identity_verified  46350 non-null  int64  
 11  host_since              46350 non-null  object 
 12  instant_bookable        46350 non-null  int64  
 13  latitude                46350 non-null  float64
 14  longitude               46350 non-null

In [16]:
# Mengubah format kolom host_since dan kolom days
df['host_since']= pd.to_datetime(df['host_since'])
df['days'] = pd.to_datetime(df['days'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46350 entries, 0 to 51878
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   property_type           46350 non-null  object        
 1   room_type               46350 non-null  object        
 2   amenities               46350 non-null  object        
 3   accommodates            46350 non-null  int64         
 4   bathrooms               46350 non-null  float64       
 5   bed_type                46350 non-null  object        
 6   cancellation_policy     46350 non-null  object        
 7   cleaning_fee            46350 non-null  int64         
 8   city                    46350 non-null  object        
 9   host_has_profile_pic    46350 non-null  int64         
 10  host_identity_verified  46350 non-null  int64         
 11  host_since              46350 non-null  datetime64[ns]
 12  instant_bookable        46350 non-null  int64 

In [17]:
# Menghitung berapa lama mereka menjadi host di airbnb
df['time'] = (df['days'] - df['host_since']).dt.days
df.head()

,property_type,room_type,amenities,accommodates,bathrooms,bed_type,cancellation_policy,cleaning_fee,city,host_has_profile_pic,...,longitude,neighbourhood,number_of_reviews,review_scores_rating,zipcode,bedrooms,beds,price,days,time
0,Apartment,Entire home/apt,"{""Wireless Internet"",""Air conditioning"",Kitche...",3,1.0,Real Bed,strict,1,NYC,1,...,-73.991617,Brooklyn Heights,2,100.0,11201,1.0,1.0,150.0,2023-02-25,3988
1,Apartment,Entire home/apt,"{""Wireless Internet"",""Air conditioning"",Kitche...",7,1.0,Real Bed,strict,1,NYC,1,...,-73.989040,Hell's Kitchen,6,93.0,10019,3.0,3.0,169.0,2023-02-25,2077
2,Apartment,Entire home/apt,"{TV,""Cable TV"",""Wireless Internet"",""Air condit...",5,1.0,Real Bed,moderate,1,NYC,1,...,-73.943756,Harlem,10,92.0,10027,1.0,3.0,145.0,2023-02-25,2314
3,House,Entire home/apt,"{TV,""Cable TV"",Internet,""Wireless Internet"",Ki...",4,1.0,Real Bed,flexible,1,SF,1,...,-122.431619,Lower Haight,0,0.0,94117.0,2.0,2.0,750.0,2023-02-25,2869
4,Apartment,Entire home/apt,"{TV,Internet,""Wireless Internet"",""Air conditio...",2,1.0,Real Bed,moderate,1,DC,1,...,-77.034596,Columbia Heights,4,40.0,20009,0.0,1.0,115.0,2023-02-25,2918


In [18]:
# Akan dihapus kolom host_since dan kolom days
# karena yang akan digunakan adalah kolom time saja
df = df.drop('host_since', axis=1)
df = df.drop('days', axis=1)
df.head()

,property_type,room_type,amenities,accommodates,bathrooms,bed_type,cancellation_policy,cleaning_fee,city,host_has_profile_pic,...,latitude,longitude,neighbourhood,number_of_reviews,review_scores_rating,zipcode,bedrooms,beds,price,time
0,Apartment,Entire home/apt,"{""Wireless Internet"",""Air conditioning"",Kitche...",3,1.0,Real Bed,strict,1,NYC,1,...,40.696524,-73.991617,Brooklyn Heights,2,100.0,11201,1.0,1.0,150.0,3988
1,Apartment,Entire home/apt,"{""Wireless Internet"",""Air conditioning"",Kitche...",7,1.0,Real Bed,strict,1,NYC,1,...,40.766115,-73.989040,Hell's Kitchen,6,93.0,10019,3.0,3.0,169.0,2077
2,Apartment,Entire home/apt,"{TV,""Cable TV"",""Wireless Internet"",""Air condit...",5,1.0,Real Bed,moderate,1,NYC,1,...,40.808110,-73.943756,Harlem,10,92.0,10027,1.0,3.0,145.0,2314
3,House,Entire home/apt,"{TV,""Cable TV"",Internet,""Wireless Internet"",Ki...",4,1.0,Real Bed,flexible,1,SF,1,...,37.772004,-122.431619,Lower Haight,0,0.0,94117.0,2.0,2.0,750.0,2869
4,Apartment,Entire home/apt,"{TV,Internet,""Wireless Internet"",""Air conditio...",2,1.0,Real Bed,moderate,1,DC,1,...,38.925627,-77.034596,Columbia Heights,4,40.0,20009,0.0,1.0,115.0,2918


In [19]:
df.to_csv('latih.csv', index = False)